In [ ]:
import networkx as nx
import networkx.algorithms.community as nx_comm
from networkx.generators.community import LFR_benchmark_graph
from networkx.algorithms import bipartite
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.sparse import coo_array
from cdlib import algorithms
from cdlib import evaluation
from utils import *
from distances import *
from consensus import *
import math
import sklearn
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.colors as mcolors

# Create graph

In [ ]:
G = nx.Graph()

G.add_nodes_from([0,1,2,3,4,5,6,7,8,9,10,11])

G.add_edge(0,1)
G.add_edge(0,2)
G.add_edge(0,3)
G.add_edge(0,4)
G.add_edge(0,6)
G.add_edge(1,2)
G.add_edge(1,3)
G.add_edge(1,5)
G.add_edge(2,3)
G.add_edge(2,8)
G.add_edge(2,11)
G.add_edge(3,4)
G.add_edge(3,5)
G.add_edge(3,7)
G.add_edge(3,9)
G.add_edge(4,5)
G.add_edge(4,6)
G.add_edge(5,6)
G.add_edge(5,7)
G.add_edge(6,7)
G.add_edge(7,8)
G.add_edge(7,9)
G.add_edge(8,10)
G.add_edge(8,11)
G.add_edge(9,11)
G.add_edge(10,11)

#pos = nx.spring_layout(G)
pos = {
    0: [1,3],
    1: [0,5],
    2: [1,9],
    3: [3,7],
    4: [4,0],
    5: [6,5],
    6: [8,1],
    7: [10,7],
    8: [1,11],
    9: [6,11],
    10: [2,14],
    11: [4,12]
    
}

fig = plt.figure(figsize=(4, 4)) # To fit in one page when printed
gs = GridSpec(nrows=1, ncols=1)
ax = fig.add_subplot(gs[0,0])

nx.draw(G, pos=pos, ax=ax, with_labels=True)
plt.savefig("toy-graph.pdf")

# Generate different clusterings

In [ ]:
# Colors chosen from https://www.color-hex.com/
YELLOW = "#f9d70b"
ORANGE = "#ff9a00"
BLACK = "#000000"
GRAY = "#999999"
LIGHT_GRAY = "#eeeeee"
LIGHT_RED = "#ffbaba"
RED = "#ff0000"
DARK_RED = "#800000"
LIGHT_BLUE = "#bbeeff"
BLUE = "#0000ff"
DARK_BLUE = "#005b96"
LIGHT_GREEN = "#c9df8a"
GREEN = "#339900"
DARK_GREEN = "#234d20"

colormap = {
    0: LIGHT_RED,
    1: LIGHT_GREEN,
    2: LIGHT_BLUE,
    3: ORANGE,
    4: RED,
    5: GREEN,
    6: BLUE,
    7: YELLOW,
    8: DARK_RED,
    9: DARK_GREEN,
    10: DARK_BLUE,
    11: LIGHT_GRAY 
}

#algs = ["infomap", "louvain", "leiden", "cnm", "label-prop", "markov", "walktrap", "spinglass"]
#algs = ["louvain", "leiden", "cnm", "label-prop", "markov", "walktrap", "spinglass"]
algs = ["louvain", "cnm", "labelprop", "markov", "walktrap"]

P_list = []
for i in range(len(algs)):
    alg = algs[i]
    if alg == "infomap":
        coms = algorithms.infomap(G)
    elif alg == "markov":
        coms = algorithms.markov_clustering(G,inflation=2.0,pruning_threshold=0.01,convergence_check_frequency=100)
    elif alg == "louvain":
        coms = algorithms.louvain(G)
    elif alg == "leiden":
        coms = algorithms.leiden(G)
    elif alg == "labelprop":
        coms = algorithms.label_propagation(G)
    elif alg == "cnm":
        coms = algorithms.greedy_modularity(G)
    elif alg == "walktrap":
        coms = algorithms.walktrap(G)
    elif alg == "spinglass":
        coms = algorithms.spinglass(G)
    print(alg, len(coms.communities))
    print(coms.communities)
    P_list.append({"graph": nx.Graph(G), "partition": list(coms.communities)})
    clust_asn = clust_lst_to_asn(coms.communities)
    node_color = []
    for n in G.nodes():
        node_color.append(colormap[clust_asn[n]])
        
    
    fig = plt.figure(figsize=(4, 4)) # To fit in one page when printed
    gs = GridSpec(nrows=1, ncols=1)
    ax = fig.add_subplot(gs[0,0])
    
    nx.draw(G, pos=pos, ax=ax, with_labels=True, node_color=node_color)
    #ax.set_title(alg)
    plt.tight_layout()
    plt.savefig("toy-graph-"+ alg +".pdf")

In [ ]:
P_list

# Generate consensus

In [ ]:
def boem_consensus(P_list, niter=10, starting_partition=None, verbose=False):
    G = nx.Graph(P_list[0]["graph"])
    n = len(list(G.nodes()))
    k = len(P_list)
    
    row = []
    col = []
    val = []
    for x in P_list:
        graph = x["graph"]
        partition = x["partition"]
        for cluster in partition:
            for i in range(len(cluster)):
                for j in range(i+1, len(cluster)):
                    item_1 = cluster[i]
                    item_2 = cluster[j]
                    row.append(min(int(item_1), int(item_2)))
                    col.append(max(int(item_1), int(item_2)))
                    val.append(int(1))
                    
    r = coo_array((val, (row, col)), shape=(n, n))
    r = r.tocsr()
    R = r.sum()
    if verbose:
        print("R:", R)
    
    rDense = r.toarray() # Should be upper triangular
    rDense = rDense + rDense.T # Making symmetric
    
    K = k - 2 * rDense
    np.fill_diagonal(K, -k) # Adding diagonal entries
    
    refined_partition = None
    if starting_partition:
        refined_partition = list(starting_partition)
    else:
        refined_partition = []
        for i in range(n):
            refined_partition.append([str(i)])
    
    refined_partition_map = clust_lst_to_map(refined_partition)
    
    items = list(refined_partition_map.keys())
    M = np.zeros((n, len(refined_partition)))
    for item_1 in items:
        for partition_id in range(len(refined_partition)):
            for item_2 in refined_partition[partition_id]:
                if(item_1 != item_2):
                    M[int(item_1),partition_id] = M[int(item_1),partition_id] + K[int(item_1), int(item_2)]
    
    mv = np.min(M, axis=1)
    mb = np.argmin(M, axis=1)
    
    count = 0
    it = 1
    while(it <= niter):
        opt_item = items[0]
        opt_deltaS = 0
        opt_a = refined_partition_map[items[0]]
        opt_b = refined_partition_map[items[0]]
        opt_x = int(opt_item)
        for item in items:
            a = refined_partition_map[item]
            x = int(item)
            b = mb[x]
            deltaS = M[x,b] - M[x,a]
            #print("Moving", item, "from", refined_partition[a], "to", refined_partition[b], "results in ", deltaS)
            if deltaS < opt_deltaS:
                opt_item = item
                opt_deltaS = deltaS
                opt_a = a
                opt_b = b
                opt_x = x
        if (opt_deltaS < 0) and (opt_a != opt_b):
            if verbose:
                print("---")
                print("Move Count:", count+1, "Optimum move results in", opt_deltaS)
                print("Move:", opt_item)
                print("From", opt_a, ":", refined_partition[opt_a])
                print("To", opt_b, ":", refined_partition[opt_b])
            for item in items:
                y = int(item)
                if y != opt_x:
                    M[y, opt_a] = M[y, opt_a] - K[y, opt_x]
                    M[y, opt_b] = M[y, opt_b] + K[y, opt_x]
            
            mv = np.min(M, axis=1)
            mb = np.argmin(M, axis=1)
            
            refined_partition[opt_a].remove(opt_item)
            refined_partition[opt_b].append(opt_item)
            refined_partition_map[opt_item] = opt_b
            if verbose:
                print("---")
            
            count = count + 1
        else:
            break

        it = it + 1
    
    empty_clusters = []
    for i in range(len(refined_partition)):
        if len(refined_partition[i]) == 0:
            empty_clusters.append(i)
            
    empty_clusters.sort(reverse=True)
    for e in empty_clusters:
        del refined_partition[e]
        
    G = nx.from_scipy_sparse_array(r)
    
    return {"graph": nx.Graph(G), "partition": list(refined_partition)}

In [ ]:
# Colors chosen from https://www.color-hex.com/
YELLOW = "#f9d70b"
ORANGE = "#ff9a00"
BLACK = "#000000"
GRAY = "#999999"
LIGHT_GRAY = "#eeeeee"
LIGHT_RED = "#ffbaba"
RED = "#ff0000"
DARK_RED = "#800000"
LIGHT_BLUE = "#bbeeff"
BLUE = "#0000ff"
DARK_BLUE = "#005b96"
LIGHT_GREEN = "#c9df8a"
GREEN = "#339900"
DARK_GREEN = "#234d20"

colormap = {
    0: LIGHT_RED,
    1: LIGHT_GREEN,
    2: LIGHT_BLUE,
    3: ORANGE,
    4: RED,
    5: GREEN,
    6: BLUE,
    7: YELLOW,
    8: DARK_RED,
    9: DARK_GREEN,
    10: DARK_BLUE,
    11: LIGHT_GRAY 
}

P_star = boem_consensus(P_list, niter=10000, starting_partition = None, verbose=False)
clust_asn = clust_lst_to_asn(P_star["partition"])

node_color = []
for n in G.nodes():
    node_color.append(colormap[clust_asn[n]])

fig = plt.figure(figsize=(4, 4)) # To fit in one page when printed
gs = GridSpec(nrows=1, ncols=1)
ax = fig.add_subplot(gs[0,0])

nx.draw(G, pos=pos, ax=ax, with_labels=True, node_color=node_color)
#ax.set_title(alg)
plt.tight_layout()
plt.savefig("toy-graph-boem.pdf")

# Set partition view

In [ ]:
# Colors chosen from https://www.color-hex.com/
YELLOW = "#f9d70b"
ORANGE = "#ff9a00"
BLACK = "#000000"
GRAY = "#999999"
LIGHT_GRAY = "#eeeeee"
LIGHT_RED = "#ffbaba"
RED = "#ff0000"
DARK_RED = "#800000"
LIGHT_BLUE = "#bbeeff"
BLUE = "#0000ff"
DARK_BLUE = "#005b96"
LIGHT_GREEN = "#c9df8a"
GREEN = "#339900"
DARK_GREEN = "#234d20"

colormap = {
    0: LIGHT_RED,
    1: LIGHT_GREEN,
    2: LIGHT_BLUE,
    3: ORANGE,
    4: RED,
    5: GREEN,
    6: BLUE,
    7: YELLOW,
    8: DARK_RED,
    9: DARK_GREEN,
    10: DARK_BLUE,
    11: LIGHT_GRAY 
}

#algs = ["infomap", "louvain", "leiden", "cnm", "label-prop", "markov", "walktrap", "spinglass"]
#algs = ["louvain", "leiden", "cnm", "label-prop", "markov", "walktrap", "spinglass"]
algs = ["louvain", "cnm", "labelprop", "markov", "walktrap"]

fig = plt.figure(figsize=(4*len(algs), 4)) # To fit in one page when printed
gs = GridSpec(nrows=1, ncols=len(algs))
axes = []
for i in range(len(algs)):
    axes.append(fig.add_subplot(gs[0,i]))

Gvset = nx.Graph(G)
Gvset.remove_edges_from(G.edges)
for i in range(len(algs)):
    alg = algs[i]
    clust_asn = clust_lst_to_asn(P_list[i]["partition"])
    node_color = []
    for n in Gvset.nodes():
        node_color.append(colormap[clust_asn[n]])
        
    nx.draw(Gvset, pos=pos, ax=axes[i], with_labels=True, node_color=node_color)
    axes[i].set_title("Set Partition: " + str(i+1))
    #axes[i].spines['right'].set_color('0.5')
    
plt.tight_layout()

# https://stackoverflow.com/questions/26084231/draw-a-separator-or-lines-between-subplots
# Get the bounding boxes of the axes including text decorations
r = fig.canvas.get_renderer()
get_bbox = lambda ax: ax.get_tightbbox(r).transformed(fig.transFigure.inverted())

print(get_bbox, np.array(axes).flat)
print("---")
print(map(get_bbox, np.array(axes).flat))
print("---")
print(list(map(get_bbox, np.array(axes).flat)))
print("---")
print(np.array(list(map(get_bbox, np.array(axes).flat))))
print("---")
print(np.array(list(map(get_bbox, np.array(axes).flat)), matplotlib.transforms.Bbox))
# print("---")
# print(np.array(list(map(get_bbox, np.array(axes).flat))).reshape(np.array(axes).shape))

# bboxes = np.array(list(map(get_bbox, np.array(axes).flat)), matplotlib.transforms.Bbox).reshape(np.array(axes).shape)
#bboxes = np.array(list(map(get_bbox, np.array(axes).flat)))


# Get the minimum and maximum extent, get the coordinate half-way between those
#ymax = np.array(list(map(lambda b: b.y1, bboxes.flat))).reshape(axes.shape).max(axis=1)
#ymin = np.array(list(map(lambda b: b.y0, bboxes.flat))).reshape(axes.shape).min(axis=1)
#ys = np.c_[ymax[1:], ymin[:-1]].mean(axis=1)

# # Draw a horizontal lines at those coordinates
# for y in ys:
#     line = plt.Line2D([0,1],[y,y], transform=fig.transFigure, color="black")
#     fig.add_artist(line)

plt.savefig("toy-graph-set-partition-view.pdf")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms as mtrans

fig, axes = plt.subplots(3,2, squeeze=False)

for i, ax in enumerate(axes.flat):
    ax.plot([1,2])
    ax.set_title('Title ' + str(i+1))
    ax.set_xlabel('xaxis')
    ax.set_ylabel('yaxis')

# rearange the axes for no overlap
fig.tight_layout()

# Get the bounding boxes of the axes including text decorations
r = fig.canvas.get_renderer()
get_bbox = lambda ax: ax.get_tightbbox(r).transformed(fig.transFigure.inverted())
bboxes = np.array(list(map(get_bbox, axes.flat)), mtrans.Bbox).reshape(axes.shape)

#Get the minimum and maximum extent, get the coordinate half-way between those
ymax = np.array(list(map(lambda b: b.y1, bboxes.flat))).reshape(axes.shape).max(axis=1)
ymin = np.array(list(map(lambda b: b.y0, bboxes.flat))).reshape(axes.shape).min(axis=1)
ys = np.c_[ymax[1:], ymin[:-1]].mean(axis=1)

# Draw a horizontal lines at those coordinates
for y in ys:
    line = plt.Line2D([0,1],[y,y], transform=fig.transFigure, color="black")
    fig.add_artist(line)


plt.show()

# Generate summary graph

In [ ]:
G = nx.Graph(P_list[0]["graph"])
n = len(list(G.nodes()))
k = len(P_list)

t1 = time.time()
row = []
col = []
val = []
for x in P_list:
    graph = x["graph"]
    partition = x["partition"]
    for cluster in partition:
        for i in range(len(cluster)):
            for j in range(i+1, len(cluster)):
                item_1 = cluster[i]
                item_2 = cluster[j]
                row.append(min(int(item_1), int(item_2)))
                col.append(max(int(item_1), int(item_2)))
                val.append(int(1))

r = coo_array((val, (row, col)), shape=(n, n))
t2 = time.time()
print("Time to prepare r:", t2-t1)
r = r.tocsr()
R = r.sum()
if verbose:
    print("R:", R)

t1 = time.time()
rDense = r.toarray() # Should be upper triangular
rDense = rDense + rDense.T # Making symmetric
t2 = time.time()
print("Time to prepare rDense:", t2-t1)

t1 = time.time()
A = nx.to_numpy_array(G)
#print(A.shape)
nz_rows, nz_cols = np.nonzero(A)
Aw = np.take(rDense, np.where(A > 0))
Aw = np.zeros(A.shape)
nz_elems = []
for i in range(len(nz_rows)):
    if rDense[nz_rows[i], nz_cols[i]] > (0.5 * k):
        nz_elems.append((nz_rows[i], nz_cols[i], rDense[nz_rows[i], nz_cols[i]]))
        Aw[nz_rows[i], nz_cols[i]] = rDense[nz_rows[i], nz_cols[i]]
nz_elems = sorted(nz_elems, key=lambda x: x[2], reverse=True)
t2 = time.time()
print("Time to filter rDense:", t2-t1)
#print(len(nz_elems))
Gw = nx.from_numpy_array(Aw)

